In [3]:
import os
import datetime
import numpy as np
from glob import glob

# ================== PARAMETRI ==================
MESSAGES_DIR = "data/messages"
ALARMS_DIR = "data/alarms"
OUT_DIR = "processed"

WINDOW_SIZE = 50
RELEVANT_DEF = (0x11, 0x18)

CAP_NOISE_MIN = 116
CAP_NOISE_MAX = 140
CAP_ACTIVE_TH = 140

FEATURES_PER_MSG = 14  # 4 time + 2 modid + 8 caps

os.makedirs(OUT_DIR, exist_ok=True)

# ================== FUNKCIJE ==================
def parse_time(ts):
    return datetime.datetime.strptime(ts, "%Y-%m-%d-%H:%M:%S:%f")

def time_features(ts):
    return [
        ts.hour / 23.0,
        ts.minute / 59.0,
        ts.second / 59.0,
        ts.microsecond / 1_000_000.0,
    ]

def normalize_caps(caps):
    out = []
    for c in caps:
        if CAP_NOISE_MIN <= c <= CAP_NOISE_MAX:
            out.append(128)
        elif c > CAP_ACTIVE_TH:
            out.append(c)
        else:
            out.append(0)
    return out

# ================== LOAD FILES ==================
message_files = sorted(glob(os.path.join(MESSAGES_DIR, "*.txt")))
print(f"Found {len(message_files)} message files")

for msg_path in message_files:
    day = os.path.basename(msg_path).replace(".txt", "")
    alarm_path = os.path.join(ALARMS_DIR, f"ALARMS-{day}.txt")

    if not os.path.exists(alarm_path):
        continue

    raw = []

    # ---------- READ MESSAGES ----------
    with open(msg_path, encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 17:
                continue
            try:
                ts = parse_time(parts[0])
                data = [int(b, 16) for b in parts[1:]]
            except:
                continue

            # filter poruka
            if data[6] not in RELEVANT_DEF:
                continue
            if data[0] != 0x03 or data[1] != 0x11:
                continue

            caps = normalize_caps(data[8:16])
            features = time_features(ts) + data[2:4] + caps
            raw.append((ts, features))

    if len(raw) < WINDOW_SIZE:
        continue

    # ---------- READ FALL EVENTS ----------
    events = []
    last_true = False

    with open(alarm_path, encoding="utf-8", errors="ignore") as f:
        for line in f:
            p = line.strip().split(",")
            if len(p) < 6:
                continue
            if p[2].strip().lower() != "fall":
                continue

            is_true = p[5].strip().lower() == "true"
            if is_true and not last_true:
                events.append(parse_time(p[0]))
            last_true = is_true

    # ---------- BUILD WINDOWS ----------
    X, Y, T = [], [], []

    for i in range(len(raw) - WINDOW_SIZE):
        window = raw[i:i + WINDOW_SIZE]
        last_ts = window[-1][0]

        label = 1 if any(
            0 <= (ev - last_ts).total_seconds() <= 1
            for ev in events
        ) else 0

        X.append([m[1] for m in window])
        Y.append(label)
        T.append(last_ts)

    X = np.asarray(X, dtype=np.float32)
    Y = np.asarray(Y, dtype=np.int64)
    T = np.asarray(T)
    EVENTS = np.asarray(events, dtype=object)

    # ---------- SAVE PER DAY ----------
    np.save(os.path.join(OUT_DIR, f"X_{day}.npy"), X)
    np.save(os.path.join(OUT_DIR, f"Y_{day}.npy"), Y)
    np.save(os.path.join(OUT_DIR, f"T_{day}.npy"), T)
    np.save(os.path.join(OUT_DIR, f"EVENTS_{day}.npy"), EVENTS, allow_pickle=True)

    print(f"{day}: windows={len(X)}, FALL events={len(events)}")

print("✅ Preprocessing finished. Data saved per day.")


Found 29 message files
2025-02-11: windows=27804, FALL events=0
2025-02-12: windows=1322, FALL events=0
2025-02-14: windows=46938, FALL events=0
2025-02-15: windows=1708, FALL events=0
2025-02-17: windows=22601, FALL events=0
2025-02-18: windows=123621, FALL events=0
2025-02-19: windows=137814, FALL events=8
2025-02-20: windows=95621, FALL events=3
2025-02-21: windows=92140, FALL events=0
2025-02-22: windows=74978, FALL events=8
2025-02-23: windows=90945, FALL events=1
2025-02-24: windows=103782, FALL events=3
2025-02-25: windows=76376, FALL events=0
2025-02-26: windows=96715, FALL events=0
2025-02-27: windows=75977, FALL events=1
2025-02-28: windows=86730, FALL events=4
2025-03-01: windows=32471, FALL events=0
2025-03-02: windows=65264, FALL events=1
2025-03-03: windows=115293, FALL events=6
2025-03-04: windows=142653, FALL events=4
2025-03-05: windows=101620, FALL events=0
2025-03-06: windows=115700, FALL events=5
2025-03-07: windows=91796, FALL events=0
2025-03-08: windows=75402, FA